In [59]:
import networkx as nx
from gnpsdata import taskresult

In [60]:
task = "cf6e14abf5604f47b28b467a513d3532"

In [61]:
# Downloading raw data from GNPS
def download_graphml(task, output_file):
    taskresult.download_task_resultfile(task, "gnps_molecular_network_graphml/", output_file)

def get_graphml_network(task):
    taskresult.download_task_resultfile(task, "gnps_molecular_network_graphml/", "temp.graphml")

    G = nx.read_graphml("temp.graphml")

    return G

def download_quantification(task, output_file):
    taskresult.download_task_resultfile(task, "quantification_table/", output_file)

def download_metadata(task, output_file):
    taskresult.download_task_resultfile(task, "metadata_merged/", output_file)

def download_mgf(task, output_file):
    taskresult.download_task_resultfile(task, "spectra_reformatted/", output_file)
    
# Qiime2 Data
def download_qiime2(task, output_file):
    taskresult.download_task_resultfile(task, "qiime2_output/qiime2_table.qza", output_file)

def download_qiime2_manifest(task, output_file):
    taskresult.download_task_resultfile(task, "qiime2_output/qiime2_manifest.tsv", output_file)

def download_qiime2_metadata(task, output_file):
    taskresult.download_task_resultfile(task, "qiime2_output/qiime2_metadata.tsv", output_file)

# Package installation

In [ ]:
!pip install pandas numpy plotly scikit-learn scikit-bio pingouin kaleido ipyfilechooser nbformat

In [120]:
# importing necessary modules
import pandas as pd
import numpy as np
import os
import itertools
import plotly.express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff
from plotly.subplots import make_subplots
from scipy.cluster.hierarchy import dendrogram, linkage
from sklearn.preprocessing import StandardScaler
from scipy.spatial import distance
from sklearn.decomposition import PCA
import scipy.stats as stats
import pingouin as pg
import skbio # Don't import on Windows!!
from ipyfilechooser import FileChooser
from ipywidgets import interact
import warnings

In [121]:
# Disable warnings for cleaner output, comment out for debugging
warnings.filterwarnings('ignore')

# ANOVA
! source activate qiime2-2019.4 && qiime longitudinal anova \
  --m-metadata-file <path-to-metadata-file.tsv> \
  --p-formula "independent_variable~dependent_variable+dependent_variable" \
  --o-visualization <path-to-visualization.qza>

In [106]:
!source activate qiime2-2019.4 && qiime longitudinal anova \
  --m-metadata-file ../data/metadata.tsv \
  --p-formula "ATTRIBUTE_Year~ATTRIBUTE_Sample_Area+ATTRIBUTE_Latitude" \
  --p-sstype 'I' \
  --o-visualization metadata.qzv

Saved Visualization to: metadata.qzv


# Visualization
Qiime2 visualizations do not work in headless environments, we can view them at https://view.qiime2.org/

In [62]:
# Qiime2 Manipulations Happen after this to mirror the other notebooks

! source activate qiime2-2019.4 && qiime

Usage: qiime [OPTIONS] COMMAND [ARGS]...

  QIIME 2 command-line interface (q2cli)
  --------------------------------------

  To get help with QIIME 2, visit https://qiime2.org.

  To enable tab completion in Bash, run the following command or add it to
  your .bashrc/.bash_profile:

      source tab-qiime

  To enable tab completion in ZSH, run the following commands or add them to
  your .zshrc:

      autoload bashcompinit && bashcompinit && source tab-qiime

Options:
  --version   Show the version and exit.
  --help      Show this message and exit.

Commands:
  info                Display information about current deployment.
  tools               Tools for working with QIIME 2 files.
  dev                 Utilities for developers and advanced users.
  alignment           Plugin for generating and manipulating alignments.
  composition         Plugin for compositional data analysis.
  cutadapt            Plugin for removing adapter sequences, primers, and
                      oth

# Principal Coordinate Analysis (PCoA) & Distance Matrix
! source activate qiime2-2019.4 && first create distance matrix 
 qiime diversity beta \
  --i-table <path-to-feature-table.qza> \
  --p-metric canberra_adkins \
  --output-dir <path-to-distance-matrix.qza> \

In [93]:
! source activate qiime2-2019.4 && qiime diversity beta \
  --i-table qiime_table.qza \
  --p-metric canberra_adkins \
  --o-distance-matrix distance_matrix.qza

Saved DistanceMatrix to: distance_matrix.qza


## PCoA
! source activate qiime2-2019.4 && qiime diversity pcoa \
  --i-distance-matrix <path-to-distance-matrix.qza> \
  --p-number-of-dimensions INTEGER \ #optional
  --o-pcoa <path-to-artifact.qza>

In [94]:
! source activate qiime2-2019.4 && qiime diversity pcoa \
  --i-distance-matrix distance_matrix.qza \
  --o-pcoa pcoa.qza

Saved PCoAResults to: pcoa.qza


# Emperor plot
! source activate qiime2-2019.4 && qiime emperor plot \
--i-pcoa <path_to_pcoa.qza> \
--m-metadata-file <path_to_metadata.tsv \
--o-visualization emperor_qiime2

In [102]:
! source activate qiime2-2019.4 && qiime emperor plot \
  --i-pcoa ./pcoa.qza \
  --m-metadata-file ./metadata.tsv \
  --o-visualization emperor_plot

Saved Visualization to: emperor_plot.qzv
